# IndicTrans

In [ ]:
!pip install setuptools==68.2.2 torch sacremoses sentencepiece transformers sacrebleu==2.3.1
!git clone https://github.com/VarunGumma/IndicTransTokenizer
!cd IndicTransTokenizer && pip install --editable ./
!git clone https://github.com/VarunGumma/indic_nlp_library.git
!pip install --editable ./indic_nlp_library
!pip install indic-nlp-library


In [ ]:
from transformers import AutoModelForSeq2SeqLM
from IndicTransTokenizer import IndicTransTokenizer
#import IndicTransTokenizer
#help(IndicTransTokenizer.processor.IndicProcessor)
#help(IndicTransTokenizer)


## (1) English to X

In [8]:
#English Dataset
with open('/kaggle/input/benchmarks689/benchmarks/wat2021-devtest/test.en', 'r') as file:
    en_dataset = file.readlines()
print(len(en_dataset))
max_length_en = max(len(line.strip()) for line in en_dataset)

2390


In [ ]:
import random
num_lines = len(en_dataset)
indices = random.sample(range(num_lines), 1000)

In [ ]:
print(indices)

In [ ]:
with open('/kaggle/working/indices_b.txt', 'w', encoding='utf-8') as file:
        for index in indices:
            file.write(str(index)+'\n')

### IF CRASHED

In [9]:
indices = []

# Read integers from the text file
with open('/kaggle/input/indices-b/indices_b.txt', 'r') as file:
    for line in file:
        indices.append(int(line.strip()))

In [10]:
sampled_en = []
for idx in indices:
    sampled_en.append(en_dataset[idx])

In [11]:
s_en = []
for sentence in sampled_en:
    words = sentence.split()
    s_en.append(words)

In [12]:
#Marathi Dataset
with open('/kaggle/input/benchmarks689/benchmarks/wat2021-devtest/test.mr', 'r') as file:
    mr_dataset = file.readlines()
print(len(mr_dataset))
max_length_mr = max(len(line.strip()) for line in mr_dataset)

2390


In [13]:
sampled_mr = []
for idx in indices:
    sampled_mr.append(mr_dataset[idx])

In [14]:
s_mr = []
for sentence in sampled_mr:
    words = sentence.split()
    s_mr.append(words)

In [ ]:
tokenizer = IndicTransTokenizer.IndicTransTokenizer(direction="en-indic")
ip = IndicTransTokenizer.processor.IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-en-indic-dist-200M", trust_remote_code=True)

In [ ]:
predictions_en_mr = []
for i in range(0,1000,50):
    batch = ip.preprocess_batch(sampled_en[i:i+50], src_lang="eng_Latn", tgt_lang="mar_Deva")
    batch = tokenizer(batch, src=True, return_tensors="pt")
    outputs = model.generate(**batch, num_beams=5, num_return_sequences=1, max_length=max_length_mr)
    outputs = tokenizer.batch_decode(outputs, src=False)
    outputs = ip.postprocess_batch(outputs, lang="mar_Deva")
    predictions_en_mr.extend(outputs)


In [ ]:
len(predictions_en_mr)

In [ ]:
with open('/kaggle/working/predictions_en_mr.txt', 'w', encoding='utf-8') as file:
        for sentence in predictions_en_mr:
            file.write(sentence.strip() + '\n')

## (2) X to English

In [ ]:
tokenizer = IndicTransTokenizer.IndicTransTokenizer(direction="indic-en")
ip = IndicTransTokenizer.processor.IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-en-dist-200M", trust_remote_code=True)

In [ ]:
predictions_mr_en = []
for i in range(0,1000,50):
    batch = ip.preprocess_batch(sampled_mr[i:i+50], src_lang="mar_Deva", tgt_lang="eng_Latn")
    batch = tokenizer(batch, src=True, return_tensors="pt")
    outputs = model.generate(**batch, num_beams=5, num_return_sequences=1, max_length=max_length_en)

    outputs = tokenizer.batch_decode(outputs, src=False)
    outputs = ip.postprocess_batch(outputs, lang="eng_Latn")
    predictions_mr_en.extend(outputs)   

In [ ]:
with open('/kaggle/working/predictions_mr_en.txt', 'w', encoding='utf-8') as file:
        for sentence in predictions_mr_en:
            file.write(sentence.strip() + '\n')

## (3) One Indian language (other than X), say Y, to X

In [ ]:
tokenizer = IndicTransTokenizer.IndicTransTokenizer(direction="indic-indic")
ip = IndicTransTokenizer.processor.IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-indic-dist-320M", trust_remote_code=True)

In [15]:
#Hindi Dataset
with open('/kaggle/input/benchmarks689/benchmarks/wat2021-devtest/test.hi', 'r') as file:
    hi_dataset = file.readlines()
#print(hi_dataset)
max_length_hi = max(len(line.strip()) for line in hi_dataset)

In [16]:
sampled_hi = []
for idx in indices:
    sampled_hi.append(hi_dataset[idx])

In [17]:
s_hi = []
for sentence in sampled_hi:
    words = sentence.split()
    s_hi.append(words)

In [ ]:
predictions_hi_mr = []
for i in range(0,1000,50):
    batch = ip.preprocess_batch(sampled_hi[i:i+50], src_lang="hin_Deva", tgt_lang="mar_Deva")
    batch = tokenizer(batch, src=True, return_tensors="pt")
    outputs = model.generate(**batch, num_beams=5, num_return_sequences=1, max_length=max_length_mr)

    outputs = tokenizer.batch_decode(outputs, src=False)
    outputs = ip.postprocess_batch(outputs, lang="mar_Deva")
    predictions_hi_mr.extend(outputs)

In [ ]:
with open('/kaggle/working/predictions_hi_mr.txt', 'w', encoding='utf-8') as file:
        for sentence in predictions_hi_mr:
            file.write(sentence.strip() + '\n')

## (4) X to Y

In [ ]:
predictions_mr_hi = []
for i in range(0,1000,50):
    batch = ip.preprocess_batch(sampled_mr[i:i+50], src_lang="mar_Deva", tgt_lang="hin_Deva")
    batch = tokenizer(batch, src=True, return_tensors="pt")
    outputs = model.generate(**batch, num_beams=5, num_return_sequences=1, max_length=max_length_hi)

    outputs = tokenizer.batch_decode(outputs, src=False)
    outputs = ip.postprocess_batch(outputs, lang="hin_Deva")
    predictions_mr_hi.extend(outputs)

In [ ]:
len(predictions_mr_hi)

In [ ]:
with open('/kaggle/working/predictions_mr_hi.txt', 'w', encoding='utf-8') as file:
        for sentence in predictions_mr_hi:
            file.write(sentence.strip() + '\n')

## Q2 : BLEU and ROUGE scores(different variants)

### IF CRASHED

In [18]:
#EN-MR
predictions_en_mr = []
with open('/kaggle/input/ques1-b/predictions_en_mr.txt', 'r') as file:
    predictions_en_mr = file.readlines()

In [19]:
#MR-EN
predictions_mr_en = []
with open('/kaggle/input/ques1-b/predictions_mr_en.txt', 'r') as file:
    predictions_mr_en = file.readlines()

In [20]:
#HI-MR
predictions_hi_mr = []
with open('/kaggle/input/ques1-b/predictions_hi_mr.txt', 'r') as file:
    predictions_hi_mr = file.readlines()

In [21]:
#MR-HI
predictions_mr_hi = []
with open('/kaggle/input/ques1-b/predictions_mr_hi.txt', 'r') as file:
    predictions_mr_hi = file.readlines()

In [22]:
!pip install evaluate
!pip install rouge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [23]:
import evaluate
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
import nltk

predictions = [predictions_en_mr, predictions_mr_en, predictions_hi_mr, predictions_mr_hi]
references = [s_mr,s_en,s_mr,s_hi]    #[sampled_mr, sampled_en, sampled_mr, sampled_hi]
tgt_lang = ['mar_Deva', 'eng_Latn', 'mar_Deva', 'hin_Deva']

bleu = evaluate.load("bleu")

for i in range(len(predictions)):
    print("Results for", tgt_lang[i], "are:\n")
    
    corpus_score = corpus_bleu(references[i], predictions[i])
    print("Corpus BLEU Score:", corpus_score, '\n')
    
    SentenceBleu = []
    
    for j in range(len(predictions[i])):
        hypothesis_tokens = nltk.word_tokenize(predictions[i][j].lower())
        reference_tokens = [nltk.word_tokenize(ref.lower()) for ref in references[i][j]]
        sentence_score = sentence_bleu(reference_tokens, hypothesis_tokens)
        SentenceBleu.append(sentence_score)
    
    AvgSentenceBleu = sum(SentenceBleu)/len(SentenceBleu)
    print("Average Sentence BLEU Score:", AvgSentenceBleu, '\n')

    results_bleu = bleu.compute(predictions=predictions[i], references=references[i])

    print(results_bleu,'\n')


2024-04-17 12:45:41.932444: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 12:45:41.932602: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 12:45:42.144902: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Results for mar_Deva are:

Corpus BLEU Score: 0.32684686250098954 



/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Average Sentence BLEU Score: 0.6168745608330305 

{'bleu': 0.006937015507671883, 'precisions': [0.46853338823690893, 0.04494878785645862, 0.0006363347120585428, 0.00017280110592707794], 'brevity_penalty': 1.0, 'length_ratio': 14.571, 'translation_length': 14571, 'reference_length': 1000} 

Results for eng_Latn are:

Corpus BLEU Score: 0.3114434670411846 

Average Sentence BLEU Score: 0.647562271776767 

{'bleu': 0.006599830159630135, 'precisions': [0.5800413412953606, 0.0403874269005848, 0.0005838090295796575, 0.0001387251161822848], 'brevity_penalty': 1.0, 'length_ratio': 17.416, 'translation_length': 17416, 'reference_length': 1000} 

Results for mar_Deva are:

Corpus BLEU Score: 0.31673429741772097 

Average Sentence BLEU Score: 0.6177154817226991 

{'bleu': 0.008691282382160498, 'precisions': [0.438811774961443, 0.0416876302738446, 0.0006194827319188478, 0.0005035246727089627], 'brevity_penalty': 1.0, 'length_ratio': 14.913, 'translation_length': 14913, 'reference_length': 1000} 



In [29]:
references = [sampled_mr, sampled_en, sampled_mr, sampled_hi]         
from rouge import Rouge
rouge = Rouge()

for i in range(len(predictions)):
    print("ROUGE Score for", tgt_lang[i], "are:")
    scores = rouge.get_scores(predictions[i], references[i],avg=True)
    print(scores,'\n')

ROUGE Score for mar_Deva are:
{'rouge-1': {'r': 0.4876732198442749, 'p': 0.48699050640378744, 'f': 0.4839405085127379}, 'rouge-2': {'r': 0.23913194003812713, 'p': 0.2382673759111858, 'f': 0.23702817363986345}, 'rouge-l': {'r': 0.461488726340589, 'p': 0.46116277667281624, 'f': 0.45811548416372283}} 

ROUGE Score for eng_Latn are:
{'rouge-1': {'r': 0.6098706349432388, 'p': 0.6134341235496755, 'f': 0.607139327578677}, 'rouge-2': {'r': 0.3606425138732938, 'p': 0.3616668721929251, 'f': 0.35820381489480907}, 'rouge-l': {'r': 0.5745465239167712, 'p': 0.577926869162274, 'f': 0.572009778675052}} 

ROUGE Score for mar_Deva are:
{'rouge-1': {'r': 0.46391745050300465, 'p': 0.4613337991192488, 'f': 0.4591363617740994}, 'rouge-2': {'r': 0.21914474170273018, 'p': 0.21753466950032788, 'f': 0.2167616266971029}, 'rouge-l': {'r': 0.43992077548246605, 'p': 0.43739337715239357, 'f': 0.4353124821302132}} 

ROUGE Score for hin_Deva are:
{'rouge-1': {'r': 0.5290764075213441, 'p': 0.5402394331094732, 'f': 0.53

In [25]:
!zip -r file.zip /kaggle/working

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


updating: kaggle/working/ (stored 0%)
updating: kaggle/working/state.db (deflated 41%)
updating: kaggle/working/indices_b.txt (deflated 54%)
updating: kaggle/working/.virtual_documents/ (stored 0%)
updating: kaggle/working/predictions_mr_en.txt (deflated 64%)
updating: kaggle/working/IndicTransTokenizer/ (stored 0%)
updating: kaggle/working/IndicTransTokenizer/LICENSE (deflated 41%)
updating: kaggle/working/IndicTransTokenizer/IndicTransTokenizer.egg-info/ (stored 0%)
updating: kaggle/working/IndicTransTokenizer/IndicTransTokenizer.egg-info/dependency_links.txt (stored 0%)
updating: kaggle/working/IndicTransTokenizer/IndicTransTokenizer.egg-info/PKG-INFO (deflated 53%)
updating: kaggle/working/IndicTransTokenizer/IndicTransTokenizer.egg-info/requires.txt (deflated 18%)
updating: kaggle/working/IndicTransTokenizer/IndicTransTokenizer.egg-info/top_level.txt (stored 0%)
updating: kaggle/working/IndicTransTokenizer/IndicTransTokenizer.egg-info/SOURCES.txt (deflated 75%)
updating: kaggle/wo

In [26]:
!ls

IndicTransTokenizer  indices_b.txt	    predictions_mr_en.txt
file.zip	     predictions_en_mr.txt  predictions_mr_hi.txt
indic_nlp_library    predictions_hi_mr.txt  state.db


In [27]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip